Finding the Feeling: Identifying the Key Emotions of a song

In [1]:
import pandas as pd
import numpy as np


Lyrics Preprocessing: To lower, 

In [11]:
import nltk
import os
import string
import re
# create new folder to hold the preprocessed data
# create new text files with the preprocessed data 
# create three functions to do upper to lower, 

rawTextDataDir = "dataset/Lyrics/"
preprocessedDataDir = "preprocessedData/Lyrics/"

#iterate over all files in rawText and create an associated proccessed file in preprocessed dir

# to lower
for fileName in os.listdir(rawTextDataDir):
    
    with open(rawTextDataDir + fileName, 'r') as data:
        newFileName = "processed" + fileName
        
        with open (preprocessedDataDir + newFileName, 'a') as newData:
            for line in data:
                newData.write(line.lower())
            newData.close();
    data.close()




In [15]:
# punctuatuon removal 

for fileName in os.listdir(preprocessedDataDir):
    with open(preprocessedDataDir + fileName, 'r') as file:
        
        for line in file:
            
            line = line.translate(str.maketrans("","", string.punctuation))
            